In [1]:
from ultralytics import YOLO

model= YOLO(r"F:\Anuj\YoloV8\YoloV8m-segmentCustom.pt")

model.predict(source = r"F:\Anuj\MyTrain\test\images\biriyanitrain-51-_jpg.rf.024286a3399bfab08065c631d0e9996a.jpg", show=False, save=True, hide_labels=False, hide_conf=False, save_txt=False, save_crop=False, line_thickness=2)

WARNING  'hide_labels' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'show_labels' instead.
WARNING  'hide_conf' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'show_conf' instead.
WARNING  'line_thickness' is deprecated and will be removed in 'ultralytics 8.399999999999999' in the future. Please use 'line_width' instead.

image 1/1 F:\Anuj\MyTrain\test\images\biriyanitrain-51-_jpg.rf.024286a3399bfab08065c631d0e9996a.jpg: 640x640 1 Biryani, 2 Boiled Eggs, 1 Chicken, 1104.9ms
Speed: 3.1ms preprocess, 1104.9ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs\segment\predict2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'Biryani', 1: 'Boiled Egg', 2: 'Chicken'}
 obb: None
 orig_img: array([[[14, 14, 14],
         [13, 13, 13],
         [12, 12, 12],
         ...,
         [34, 30, 25],
         [35, 31, 26],
         [35, 31, 26]],
 
        [[15, 15, 15],
         [15, 15, 15],
         [15, 15, 15],
         ...,
         [34, 30, 25],
         [35, 31, 26],
         [36, 32, 27]],
 
        [[ 9,  9,  9],
         [10, 10, 10],
         [11, 11, 11],
         ...,
         [34, 30, 25],
         [35, 31, 26],
         [36, 32, 27]],
 
        ...,
 
        [[ 2,  3,  1],
         [ 2,  3,  1],
         [ 2,  3,  1],
         ...,
         [23, 14, 11],
         [24, 15, 12],
         [26, 17, 14]],
 
        [[ 2,  3,  1],
         [ 2,  3,  1],
         [ 2,  3,  1],
         ...,
         [23, 14, 11],
         

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO(r"F:\Anuj\YoloV8\YoloV8m-segmentCustom.pt")  # Replace with the path to your YOLOv8 model

def display_and_segment_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Run YOLOv8 model
    results = model.predict(image)
    
    # Extract bounding boxes and class labels
    for result in results:
        for bbox in result.boxes:
            x1, y1, x2, y2 = map(int, bbox.xyxy[0])
            label = result.names[int(bbox.cls[0])]
            
            # Draw bounding box and label on the image
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    
    # Display the image with bounding boxes and labels
    cv2.imshow('Detected Image', image)
    cv2.waitKey(0)
    
    # Perform segmentation on each ROI
    for result in results:
        for bbox in result.boxes:
            x1, y1, x2, y2 = map(int, bbox.xyxy[0])
            roi = image[y1:y2, x1:x2]
            
            # Convert ROI to grayscale
            gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            
            # Apply thresholding
            _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            
            # Find contours
            contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Create mask from contours
            mask = np.zeros(roi.shape[:2], np.uint8)
            cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
            
            # Create segmented image using mask
            segmented_image = cv2.bitwise_and(roi, roi, mask=mask)
            
            # Insert the segmented ROI back into the original image
            image[y1:y2, x1:x2] = segmented_image
    
    # Save or display the segmented image
    cv2.imwrite('segmented_image.jpg', image)
    cv2.imshow('Segmented Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
display_and_segment_image(r"F:\Anuj\MyTrain\test\images\biriyanitrain-29-_jpg.rf.53d7420e3d0ebee18266c15afdc43899.jpg")



0: 640x640 1 Biryani, 1 Boiled Egg, 1 Chicken, 1091.5ms
Speed: 5.2ms preprocess, 1091.5ms inference, 4.2ms postprocess per image at shape (1, 3, 640, 640)


In [4]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO(r"F:\Anuj\YoloV8\YoloV8m-segmentCustom.pt")  # Replace with the path to your YOLOv8 model

def display_segmentation(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Run YOLOv8 model
    results = model.predict(image, task='segment')  # Ensure task is set to 'segment'
    
    # Extract segmentation masks
    for result in results:
        if result.masks is not None:
            for mask in result.masks.data:
                # Convert mask to a format suitable for overlay
                mask = mask.cpu().numpy()
                seg_mask = (mask * 255).astype(np.uint8)

                # Apply the mask to the image
                color_mask = np.zeros_like(image)
                color_mask[seg_mask == 255] = [0, 255, 0]  # Green color for the mask area

                # Blend the color mask with the original image
                image = cv2.addWeighted(image, 1, color_mask, 0.5, 0)
    
    # Display the segmented image
    cv2.imshow('Segmented Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
display_segmentation(r"F:\Anuj\MyTrain\test\images\biriyanitrain-51-_jpg.rf.024286a3399bfab08065c631d0e9996a.jpg")



0: 640x640 1 Biryani, 2 Boiled Eggs, 1 Chicken, 1088.5ms
Speed: 17.2ms preprocess, 1088.5ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 model
model = YOLO(r"F:\Anuj\YoloV8\YoloV8m-segmentCustom.pt")  # Replace with the path to your YOLOv8 model

def display_segmentation(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Run YOLOv8 model
    results = model.predict(image, task='segment')  # Ensure task is set to 'segment'
    
    # Create a blank mask for the image
    final_mask = np.zeros(image.shape[:2], dtype=np.uint8)
    
    # Get the class names from the model
    class_names = model.names

    # Extract segmentation masks and draw bounding boxes and labels
    for result in results:
        if result.masks is not None:
            for mask, box in zip(result.masks.data, result.boxes):
                # Convert mask to a format suitable for overlay
                mask = mask.cpu().numpy()
                seg_mask = (mask * 255).astype(np.uint8)

                # Combine masks to create a final mask
                final_mask = cv2.bitwise_or(final_mask, seg_mask)

                # Draw the bounding box
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Put the class label
                class_id = int(box.cls)
                class_name = class_names[class_id]
                cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Apply the final mask to the image
    segmented_image = cv2.bitwise_and(image, image, mask=final_mask)

    # Display the original image with bounding boxes and labels
    cv2.imshow('Predicted Image', image)
    
    # Display the segmented image
    cv2.imshow('Segmented Image', segmented_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Save the segmented image if needed
    cv2.imwrite('segmented_image.jpg', segmented_image)

# Example usage
display_segmentation(r"F:\Anuj\MyTrain\test\images\biriyanitrain-48-_jpg.rf.5f9f98b31ce9725190e56f93da2d2354.jpg")



0: 640x640 1 Biryani, 1143.5ms
Speed: 3.2ms preprocess, 1143.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
